In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

# GMM-related functions

In [ ]:
# Function to compute the likelihood of a GMM
def gmm_lik(x, pi, mu, sigma):
    """Calculates the probability density of a GMM."""
    k = len(pi)
    density = np.zeros_like(x)
    for i in range(k):
        density += ???
    return density

# Function to compute the log-likelihood of a GMM
def gmm_log_lik(x, pi, mu, sigma):
    """Calculates the log-likelihood of a GMM."""
    likelihood = gmm_lik(x, pi, mu, sigma)
    return np.sum(np.log(likelihood + 1e-10))  # Adding a small constant to avoid log(0)

# Function to compute the posterior probabilities of the GMM
def gmm_posterior_probabilities(x, pi, mu, sigma):
    """Calculates the posterior probabilities of a GMM."""
    k = len(pi)
    likelihood = gmm_lik(x, pi, mu, sigma)
    posteriors = np.zeros((len(x), k))
    for i in range(k):
        posteriors[:, i] = pi[i] * multivariate_normal.pdf(x, mu[i], sigma[i])
    posteriors /= likelihood[:, np.newaxis]
    return posteriors

# Function to sample from a GMM
def gmm_sample(pi, mu, sigma, n_samples):
    """Samples from a GMM."""
    k = len(pi)
    z = ??
    x = ??
    return x



# GMM inference algorithms

In [ ]:
# Function to run the k-means algorithm (i.e., the same as above but with hard assignments) for GMM
def kmeans_gmm(x, k, max_iters=100, tol=1e-6):
    """Performs k-means algorithm for GMM."""
    n, d = x.shape
    pi = np.ones(k) / k
    mu = x[np.random.choice(n, k, replace=False)]
    sigma = [np.eye(d) for _ in range(k)]

    # Initialize lists to store log-likelihoods and ELBOs
    log_likelihoods = []

    # Iterate until max iterations or convergence, given tolerance
    for iteration in range(max_iters):
        # Responsibility step
        responsibilities = np.zeros((n, k))
        ???
        
        # Assign each point to the cluster with the highest responsibility
        z_n = np.argmax(responsibilities, axis=1)

        # Update the parameters
        nk = np.bincount(z_n, minlength=k)
        pi = nk / n
        mu = np.array([x[z_n == i].mean(axis=0) for i in range(k)])
        for i in range(k):
            diff = x[z_n == i] - mu[i]
            sigma[i] = np.dot(diff.T, diff) / nk[i]
        
        # Calculate Loglikelihood of assigned clusters and data, given the parameters.
        log_likelihood = np.sum(np.log(np.sum([pi[j] * multivariate_normal.pdf(xi, mu[j], sigma[j]) for j in range(k)], axis=0)) for xi in x)
        log_likelihoods.append(log_likelihood)

        # Plotting
        plt.figure(figsize=(12, 6))

        # The current estimated GMM
        plt.scatter(x[:, 0], x[:, 1], alpha=0.5)
        for i in range(k):
            # Plot the mean of each cluster
            plt.scatter(mu[i, 0], mu[i, 1], marker='x', s=100, c='orange', label=f'Cluster {i+1} mean')
            # Draw ellipse for covariance matrix for 2-D case
            if d == 2:
                from matplotlib.patches import Ellipse
                eigenvalues, eigenvectors = np.linalg.eigh(sigma[i])
                angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
                width, height = 2 * np.sqrt(eigenvalues)
                ell = Ellipse(xy=mu[i], width=width, height=height, angle=angle, edgecolor='orange', fc='None', lw=2, label=f'Cluster {i+1} covariance')
                plt.gca().add_patch(ell)
        plt.title(f"K-Means Iteration {iteration + 1} - Estimated GMM Parameters")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.xlim(np.min(x[:, 0]), np.max(x[:, 0]))
        plt.ylim(np.min(x[:, 1]), np.max(x[:, 1]))
        plt.legend()
        plt.grid()
        plt.tight_layout()
        plt.show()

        # Check for convergence
        if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            break
    
    # Plotting log-likelihood over iterations
    plt.figure(figsize=(8,6))
    plt.plot(log_likelihoods)
    plt.title("K-means Log likelihood over iterations")
    plt.xlabel("Iteration")
    plt.ylabel("Log likelihood")
    plt.show()
    
    return pi, mu, sigma, log_likelihoods

In [ ]:
# Function to run the EM algorithm for GMM
def em_gmm(x, k, max_iters=100, tol=1e-6):
    """Performs EM algorithm for GMM."""
    n, d = x.shape
    pi = np.ones(k) / k
    mu = x[np.random.choice(n, k, replace=False)]
    sigma = [np.eye(d) for _ in range(k)]

    # Initialize lists to store log-likelihoods and ELBOs
    log_likelihoods = []
    elbos = []
    kl_divergences = []

    # Iterate until max iterations or convergence, given tolerance
    for iteration in range(max_iters):
        # E-step
        responsibilities = np.zeros((n, k))
        ??

        # M-step
        nk = responsibilities.sum(axis=0)
        pi = nk / n
        mu = np.dot(responsibilities.T, x) / nk[:, np.newaxis]
        for i in range(k):
            diff = x - mu[i]
            sigma[i] = np.dot(responsibilities[:, i] * diff.T, diff) / nk[i]

        # Calculate ELBO
        elbo = 0
        for i in range(n):
            for j in range(k):
                if responsibilities[i,j] >0:
                    elbo += responsibilities[i, j] * (np.log(pi[j]) + multivariate_normal.logpdf(x[i], mu[j], sigma[j]) - np.log(responsibilities[i,j]))
        elbos.append(elbo)

        #Calculate Loglikelihood.
        log_likelihood = np.sum(np.log(np.sum([pi[j] * multivariate_normal.pdf(xi, mu[j], sigma[j]) for j in range(k)], axis=0)) for xi in x)
        log_likelihoods.append(log_likelihood)

        kl_divergence = log_likelihood - elbo
        kl_divergences.append(kl_divergence)

        # Plotting
        plt.figure(figsize=(12, 6))

        # The current estimated GMM
        plt.scatter(x[:, 0], x[:, 1], alpha=0.5)
        for i in range(k):
            plt.scatter(mu[i, 0], mu[i, 1], marker='x', s=100, c='green', label=f'Cluster {i+1} mean')
            # Draw ellipse for covariance matrix for 2-D case
            if d == 2:
                from matplotlib.patches import Ellipse
                eigenvalues, eigenvectors = np.linalg.eigh(sigma[i])
                angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
                width, height = 2 * np.sqrt(eigenvalues)
                ell = Ellipse(xy=mu[i], width=width, height=height, angle=angle, edgecolor='green', fc='None', lw=2, label=f'Cluster {i+1} covariance')
                plt.gca().add_patch(ell)
        plt.title(f"EM Iteration {iteration + 1} - Estimated GMM")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.xlim(np.min(x[:, 0]), np.max(x[:, 0]))
        plt.ylim(np.min(x[:, 1]), np.max(x[:, 1]))
        plt.legend()
        plt.grid()
        plt.tight_layout()
        plt.show()

        if iteration > 0 and abs(elbos[-1] - elbos[-2]) < tol:
            break

    # Plotting ELBO and log-likelihood over iterations
    plt.figure(figsize=(8,6))
    plt.plot(log_likelihoods, label='Log Likelihood')
    plt.plot(elbos, label='ELBO')
    plt.title("EM: Log Likelihood and ELBO over iterations")
    plt.xlabel("Iteration")
    plt.ylabel("Value")
    plt.legend()
    plt.show()

    # Plotting KL divergence
    plt.figure(figsize=(8,6))
    plt.plot(kl_divergences)
    plt.title("EM: KL Divergence between ELBO and Log likelihood over iterations")
    plt.xlabel("Iteration")
    plt.ylabel("KL Divergence")
    plt.show()

    return pi, mu, sigma, log_likelihoods, elbos, kl_divergences

# Generate GMM and draw from it

In [ ]:
# Set random seed for reproducibility, if needed
# Define true parameters
true_pi = [0.3, 0.7]
true_mu = [[1, 1], [-1, -1]]
true_sigma = [[[1, 0], [0, 1]], [[0.5, 0], [0, 0.5]]]
n_samples = 500
k = 2  # Number of components

# Generate synthetic data
x = gmm_sample(true_pi, true_mu, true_sigma, n_samples)

# Plot the generated data
plt.scatter(x[:, 0], x[:, 1], alpha=0.5)
plt.title("Generated Data from GMM")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()

## Run k-means algorithm


In [ ]:
# Run the k-means algorithm
kmeans_pi, kmeans_mu, kmeans_sigma, kmeans_log_likelihoods = kmeans_gmm(
    x,
    k,
    max_iters=100,
    tol=1e-6
)

### Can you modify the k-means algorithm to plot the assigned clusters to each data point?

## Run the EM algorithm

In [ ]:
# Run the EM algorithm
em_pi, em_mu, em_sigma, em_log_likelihoods, em_elbos, em_kl_divergences = em_gmm(
    x,
    k,
    max_iters=100,
    tol=1e-6
)

## Compare the results for K-means and EM

In [ ]:
# Compute parameter errors
# For k-means
kmeans_pi_error = np.abs(np.array(kmeans_pi) - np.array(true_pi))
kmeans_mu_error = np.array(kmeans_mu) - np.array(true_mu)
kmeans_sigma_error = np.array(kmeans_sigma) - np.array(true_sigma)
# For EM
em_pi_error = np.abs(np.array(em_pi) - np.array(true_pi))
em_mu_error = np.array(em_mu) - np.array(true_mu)
em_sigma_error = np.array(em_sigma) - np.array(true_sigma)

# Compare the estimated parameters with the true parameters
print("True Mixing Coefficients:", true_pi)
print("K-means Estimated Mixing Coefficients:", kmeans_pi)
print("\t_K-means Mixing Coefficient Errors:", kmeans_pi_error)
print("EM Estimated Mixing Coefficients:", em_pi)
print("\tEM Mixing Coefficient Errors:", em_pi_error)

print("True Means:", true_mu)
print("K-means Estimated Means:", kmeans_mu)
print("\tK-means Mean Errors:", kmeans_mu_error)
print("EM Estimated Means:", em_mu)
print("\tEM Mean Errors:", em_mu_error)

print("True Covariances:", true_sigma)
print("K-means Estimated Covariances:", kmeans_sigma)
print("\tK-means Covariance Errors:", kmeans_sigma_error)
print("EM Estimated Covariances:", em_sigma)
print("\tEM Covariance Errors:", em_sigma_error)

In [ ]:
# Plot the estimated parameters
plt.figure(figsize=(12, 6))
plt.scatter(x[:, 0], x[:, 1], alpha=0.5)
for i in range(k):
    # Plot estimated means
    # K-means
    plt.scatter(kmeans_mu[i][0], kmeans_mu[i][1], marker='x', s=100, c='orange')
    # EM 
    plt.scatter(em_mu[i][0], em_mu[i][1], marker='x', s=100, c='green')
    # Plot true means
    plt.scatter(true_mu[i][0], true_mu[i][1], marker='x', s=100, c='red')

    # Plot estimated covariance matrices
    # Draw ellipse for covariance matrix for 2-D case
    if len(true_mu[i]) == 2:
        from matplotlib.patches import Ellipse
        # For estimated covariance matrix
        # K-means
        eigenvalues, eigenvectors = np.linalg.eigh(kmeans_sigma[i])
        angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
        width, height = 2 * np.sqrt(eigenvalues)
        ell = Ellipse(xy=kmeans_mu[i], width=width, height=height, angle=angle, edgecolor='orange', fc='None', lw=2)
        plt.gca().add_patch(ell)
        # EM
        eigenvalues, eigenvectors = np.linalg.eigh(em_sigma[i])
        angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
        width, height = 2 * np.sqrt(eigenvalues)
        ell = Ellipse(xy=em_mu[i], width=width, height=height, angle=angle, edgecolor='green', fc='None', lw=2)
        plt.gca().add_patch(ell)
        # For true covariance matrix
        eigenvalues, eigenvectors = np.linalg.eigh(true_sigma[i])
        angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
        width, height = 2 * np.sqrt(eigenvalues)
        ell = Ellipse(xy=true_mu[i], width=width, height=height, angle=angle, edgecolor='red', fc='None', lw=2)
        plt.gca().add_patch(ell)

plt.title("Estimated GMM Parameters")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()

In [ ]:
# Plot the log-likelihoods
plt.figure(figsize=(12, 6))
plt.plot(kmeans_log_likelihoods, label='K-means Log Likelihood', color='orange')
plt.plot(em_log_likelihoods, label='EM Log Likelihood', color='green')
plt.title("Log Likelihood over iterations")
plt.xlabel("Iteration")
plt.ylabel("Log Likelihood")
plt.legend()
plt.show()

# Print the final log-likelihoods
print("Final K-means Log Likelihood:", kmeans_log_likelihoods[-1])
print("Final EM Log Likelihood:", em_log_likelihoods[-1])

### How do the above algorithms change 

- with different true GMM parameters?

- with different initializations?

- with different number of samples?

- with different number of clusters?
